In [1]:
import asyncio
import pandas as pd
import numpy as np
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json

In [2]:
test_data = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [3]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COMPETITORS_DETAILS
blob_shipping_cost = settings.BLOB_COMPETITORS_DETAILS(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-09-20
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_0__process_time=2024-09-20T12:22:22.658330-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_1__process_time=2024-09-20T12:23:23.953020-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_2__process_time=2024-09-20T12:23:23.222408-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_3__process_time=2024-09-20T12:23:23.708533-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_d

In [15]:
json = content[0]

dict_content = {
                        'item_id' : json.get('item_id'),
                        'price_id': json.get('site_id'),
                        'competitors_type': 'input',
                        'category_id': json.get('category_id'),
                        'official_store_id': json.get('official_store_id'),
                        'initial_quantity': json.get('initial_quantity'),
                        'competitor_seller_id': json.get('seller_id'),
                        'buying_mode': json.get('buying_mode'),
                        'listing_type_id': json.get('listing_type_id'),
                        'condition': json.get('condition'),
                        'permalink': json.get('permalink'),
                        'status' : json.get('status'),
                        }


dict_content      

{'item_id': 'MLB3651481701',
 'price_id': 'MLB',
 'competitors_type': 'input',
 'category_id': 'MLB189908',
 'official_store_id': None,
 'initial_quantity': 21,
 'competitor_seller_id': 1207642281,
 'buying_mode': 'buy_it_now',
 'listing_type_id': 'gold_special',
 'condition': 'new',
 'permalink': 'https://produto.mercadolivre.com.br/MLB-3651481701-controlador-inteligente-fita-led-cct-wi-fible-alexa-google-_JM',
 'status': 'active'}

In [5]:
content[0]

{'id': 'MLB3651481701',
 'site_id': 'MLB',
 'title': 'Controlador Inteligente Fita Led Cct Wi-fi+ble Alexa Google',
 'seller_id': 1207642281,
 'category_id': 'MLB189908',
 'official_store_id': None,
 'price': 77.17,
 'base_price': 77.17,
 'original_price': None,
 'currency_id': 'BRL',
 'initial_quantity': 21,
 'sale_terms': [{'id': 'WARRANTY_TIME',
   'name': 'Tempo de garantia',
   'value_id': None,
   'value_name': '12 meses',
   'value_struct': {'number': 12, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '12 meses',
     'struct': {'number': 12, 'unit': 'meses'}}],
   'value_type': 'number_unit'},
  {'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantia',
   'value_id': '2230279',
   'value_name': 'Garantia de fábrica',
   'value_struct': None,
   'values': [{'id': '2230279',
     'name': 'Garantia de fábrica',
     'struct': None}],
   'value_type': 'list'}],
 'buying_mode': 'buy_it_now',
 'listing_type_id': 'gold_special',
 'condition': 'new',
 'permalink': 'https://prod

In [6]:
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
store_name = 'hubsmarthome'
blob_path = f'{store_name}/meli/api_response/orders/'


storage.blob_exists(bucket_name= 'glm-store', blob_name=blob_path)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


False

In [139]:
test_data = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [43]:
async def process_date(date, storage, bucket_name, blob_shipping_cost):
    try:
        print(f'Processing date: {date}')
        blob_prefix = blob_shipping_cost + f'date={date}/'

        # Use run_in_executor to call synchronous methods without blocking
        loop = asyncio.get_event_loop()
        blobs = await loop.run_in_executor(None, storage.list_blobs, bucket_name, blob_prefix)

        df_processed_data = pd.DataFrame()

        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = await loop.run_in_executor(None, storage.download_json, bucket_name, blob.name)

            for json_content in content:
                df_ = await process_orders(json_content['results'])
                df_processed_data = pd.concat([df_processed_data, df_], ignore_index=True)

        if not df_processed_data.empty:
            df_processed_data['processed_json'] = pd.to_datetime(date)
            df_processed_data['process_time'] = datetime.now()

        print(f'*** Finished treating all data for date {date}. {df_processed_data.shape[0]} sales ***')

        return df_processed_data
    except Exception as e:
        print(f'Error processing date {date}: {e}')
        return pd.DataFrame()

def process_orders(json_data):
    try:
        df_ = pd.DataFrame()

        for sale in json_data:
            structured_sale = {
                # Your structured_sale dictionary here
            }

            # Create a new DataFrame for the structured sale
            df_sale = pd.DataFrame([structured_sale])

            # Concatenate only if df_ is not empty
            if df_.empty:
                df_ = df_sale  # Assign the first entry directly if df_ is empty
            else:
                df_ = pd.concat([df_, df_sale], ignore_index=True)

        return df_
    except Exception as e:
        print(f'Error processing json: {e}')
        return pd.DataFrame()



In [24]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')

print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_ORDERS
blob_shipping_cost = settings.BLOB_ORDERS(store_name)
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
list_dates_to_process = [date.strftime('%Y-%m-%d') for date in list_dates_to_process]
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
# Use asyncio.gather to process dates asynchronously
tasks = [process_date(date, storage, bucket_name, blob_shipping_cost) for date in list_dates_to_process]
results = await asyncio.gather(*tasks)
# Combine all DataFrames
df_all_processed_data = pd.concat(results, ignore_index=True)
print(f'*** Finished processing all dates. Total sales: {df_all_processed_data.shape[0]} ***')

In [47]:
date = list_dates_to_process[3]
print(f'Processing date: {date}')
blob_prefix = blob_shipping_cost + f'date={date}/'
# Use run_in_executor to call synchronous methods without blocking
loop = asyncio.get_event_loop()
blobs = await loop.run_in_executor(None, storage.list_blobs, bucket_name, blob_prefix)
df_processed_data = pd.DataFrame()
for blob in blobs:
    print(f"Reading file: {blob.name}")
    content = await loop.run_in_executor(None, storage.download_json, bucket_name, blob.name)
    for json_content in content:
        df_ = process_orders(json_content['results'])
        df_processed_data = pd.concat([df_processed_data, df_], ignore_index=True)
if not df_processed_data.empty:
    df_processed_data['processed_json'] = pd.to_datetime(date)
    df_processed_data['process_time'] = datetime.now()
print(f'*** Finished treating all data for date {date}. {df_processed_data.shape[0]} sales ***')

df_processed_data

Processing date: 2024-04-27
Reading file: hubsmarthome/meli/api_response/orders/date=2024-04-27/total_sales=384__data=2024-04-27__processing-time=2024-09-17T17:42:42.561756-03:00.json
*** Finished treating all data for date 2024-04-27. 8 sales ***


""
0
1
2
3
4
5
6
7


In [52]:
df_ = pd.DataFrame()
for sale in json_content:
    structured_sale = {
        # Your structured_sale dictionary here
    }
    # Create a new DataFrame for the structured sale
    df_sale = pd.DataFrame([structured_sale])
    # Concatenate only if df_ is not empty
    if df_.empty:
        df_ = df_sale  # Assign the first entry directly if df_ is empty
    else:
        df_ = pd.concat([df_, df_sale], ignore_index=True)

structured_sale

{}

In [44]:
process_orders(json_content['results'])

""
0


# Fetch historical orders

In [26]:
import aiohttp
import asyncio
from datetime import datetime, timedelta
import pandas as pd
import json
import aiofiles
from concurrent.futures import ThreadPoolExecutor
from flask import escape
import requests
import gc  # Garbage collection to free memory
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import log_process, authenticate
from src.config import settings

In [27]:
timezone_offset = "-03:00"
limit = 50

def fetch_data(date, offset, access_token, seller_id):
    formatted_date_start = date.strftime(f"%Y-%m-%dT00:00:00.000{timezone_offset}")
    formatted_date_end = (date + timedelta(days=1)).strftime(f"%Y-%m-%dT00:00:00.000{timezone_offset}")
    params = {
        'limit': limit,
        'offset': offset,
        'order.date_created.from': formatted_date_start,
        'order.date_created.to': formatted_date_end
    }
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    url = settings.URL_ORDERS(seller_id)

    response = requests.get(url, params=params, headers=headers) 
    print(response.json)
    return response

In [28]:
data = test_data
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_ORDERS
start_date = datetime.today() - timedelta(days=365)
end_date = datetime.today() - timedelta(days=1)
blob_name = settings.BLOB_ORDERS(store_name)

In [29]:
offset = 0
responses = []
sales = []
date = end_date
# while True:
response = fetch_data(date, offset, access_token, seller_id)

response.json()

In [30]:
response

In [55]:
# Provided dictionary keys from the code
code_keys = [
    'reason', 'status_code', 'total_paid_amount', 'operation_type', 'transaction_amount', 
    'transaction_amount_refunded', 'date_approved', 'collector_id', 'coupon_id', 'installments',
    'authorization_code', 'taxes_amount', 'payment_id', 'date_last_modified', 'coupon_amount', 
    'payment_shipping_cost', 'installment_amount', 'activation_uri', 'overpaid_amount', 'card_id', 
    'payment_status_detail', 'issuer_id', 'payment_method_id', 'payment_type', 'deferred_period', 
    'atm_transfer_reference_transaction_id', 'atm_transfer_reference_company_id', 'site_id', 
    'payer_id', 'order_id', 'currency_id', 'payment_status', 'shipping_id', 'fulfilled', 'seller_id', 
    'buyer_id', 'item_id', 'item_title', 'item_category_id', 'item_variation_id', 'seller_custom_field', 
    'global_price', 'net_weight', 'warranty', 'condition', 'seller_sku', 'quantity', 'unit_price', 
    'full_unit_price', 'manufacturing_days', 'requested_quantity_measure', 'requested_quantity_value', 
    'sale_fee', 'listing_type_id', 'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id', 
    'date_created', 'date_closed', 'status', 'expiration_date', 'date_last_updated', 'last_updated', 
    'comment', 'pack_id', 'coupon_amount', 'coupon_id', 'shipping_cost', 'pickup_id', 'status_detail', 
    'total_amount', 'paid_amount', 'context_application', 'context_product_id', 'context_channel', 
    'context_site'
]

# Keys from the schema
schema_keys = [
    'reason', 'status_code', 'total_paid_amount', 'operation_type', 'transaction_amount',
    'transaction_amount_refunded', 'date_approved', 'collector_id', 'coupon_id', 'installments',
    'authorization_code', 'taxes_amount', 'payment_id', 'date_last_modified', 'coupon_amount', 
    'shipping_cost', 'installment_amount', 'activation_uri', 'overpaid_amount', 'card_id', 
    'status_detail', 'issuer_id', 'payment_method_id', 'payment_type', 'deferred_period', 
    'atm_transfer_reference_transaction_id', 'atm_transfer_reference_company_id', 'site_id', 
    'payer_id', 'order_id', 'currency_id', 'payment_status', 'shipping_id', 'fulfilled', 'seller_id', 
    'buyer_id', 'item_id', 'item_title', 'item_category_id', 'item_variation_id', 'seller_custom_field', 
    'global_price', 'net_weight', 'warranty', 'condition', 'seller_sku', 'quantity', 'unit_price', 
    'full_unit_price', 'manufacturing_days', 'requested_quantity_measure', 'requested_quantity_value', 
    'sale_fee', 'listing_type_id', 'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id', 
    'date_created', 'date_closed', 'status', 'expiration_date', 'date_last_updated', 'last_updated', 
    'comment', 'pack_id', 'pickup_id', 'total_amount', 'paid_amount', 'context_application', 
    'context_product_id', 'context_channel', 'context_site', 'processed_json', 'process_time'
]

# Find keys that are in the dictionary but not in the schema
extra_keys = set(code_keys) - set(schema_keys)
extra_keys = set(schema_keys) - set(code_keys)  

extra_keys

{'process_time', 'processed_json'}

In [58]:
extra_keys_ = set(code_keys) - set(schema_keys)
extra_keys = set(schema_keys) - set(code_keys)  
extra_keys

{'process_time', 'processed_json'}

In [59]:
extra_keys_

{'payment_shipping_cost', 'payment_status_detail'}

In [17]:
def authenticate(client_id, client_secret):
  url = "https://api.mercadolibre.com/oauth/token"
  payload = {
      'grant_type': 'client_credentials',
      'client_id': client_id,
      'client_secret': client_secret
  }
  response = requests.post(url, data=payload)
  if response.status_code == 200:
    return response.json()['access_token']
  else:
    raise Exception("Authentication failed")

In [29]:
import requests

client_id     = "4959083987776428"
client_secret = "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw"

access_token = authenticate(client_id, client_secret)

In [172]:
URL_SHIPPING_COST = f'https://api.mercadolibre.com/users/189643563/shipping_options/free?item_id={item_id}'
URL_SHIPPING_COST

In [197]:
item_id = 'MLB4237237960'
# url = f'https://api.mercadolibre.com/items/{item_id}/sale_price?context=channel_marketplace, channel_mshops'
url =  f'https://api.mercadolibre.com/users/189643563/shipping_options/free?item_id={item_id}&channel=mshops'

headers = {
        'Authorization': f'Bearer {access_token}'
    }
response = requests.get(url, headers=headers)
data = response.json()
data

{'coverage': {'all_country': {'list_cost': 21.45,
   'currency_id': 'BRL',
   'billable_weight': 750}}}

In [187]:
url = 'https://api.mercadolibre.com/sites/MLB/listing_prices?price=500&listing_type_id=gold_pro'
response = requests.get(url, headers=headers)
response.json()

[{'currency_id': 'BRL',
  'free_relist': False,
  'listing_exposure': 'highest',
  'listing_fee_amount': 0,
  'listing_fee_details': {'fixed_fee': 0, 'gross_amount': 0},
  'listing_type_id': 'gold_pro',
  'listing_type_name': 'Premium',
  'requires_picture': True,
  'sale_fee_amount': 80,
  'sale_fee_details': {'fixed_fee': 0,
   'gross_amount': 80,
   'percentage_fee': 16},
  'stop_time': '2044-09-15T00:00:00.000-04:00'}]

# Getting costs for mlshops

In [198]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [199]:
data = test_data
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COSTS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
blob_prices = settings.BLOB_PRICES(store_name)

items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names=['id','price', 'category_id', 'listing_type_id']
)

prices = fetch_items_from_storage(
storage, 
bucket_name, 
blob_prices, 
key_names=['id','price', 'category_id', 'listing_type_id']
)



** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=0__total_items=100__process_time=2024-09-20T09:10:10.048471-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=1__total_items=100__process_time=2024-09-20T09:10:10.942620-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=2__total_items=100__process_time=2024-09-20T09:10:10.829410-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=3__total_items=100__process_time=2024-09-20T09:10:10.931497-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=4__

In [200]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json

In [201]:
test_data = {
  "access_token": "APP_USR-2951712600123976-092103-9ed59662b4bd7dfa1e8eb085d50e677c-150350803",
  "client_id": "2951712600123976",
  "client_secret": "QprAIl8ydXzcxFVHjnIHT6fUQ8KpzADV",
  "seller_id": 150350803,
  "store_name": "berga eletro_pr"
}

In [202]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_PRICES
blob_shipping_cost = settings.BLOB_PRICES(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 0 dates to process  ***


In [203]:
date_to_process = '2024-09-21'
print(f'Processing date: {date_to_process}')
# Get blob with the date
blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
# List all the files
blobs = storage.list_blobs(bucket_name, blob_prefix)
content = storage.download_json(bucket_name, blob.name)

Processing date: 2024-09-21


In [247]:
def process_prices(json):

    try:
        extracted_data = []

        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}

        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]


            # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
            if channel not in price_by_channel or price['type'] == 'promotion':
                price_by_channel[channel] = {
                    'item_id': json.get('id'),
                    'price_id': price.get('id'),
                    'regular_amount': price.get('regular_amount'),
                    'price': price.get('amount'),
                    'channel': channel,
                    'last_updated': price.get('last_updated')
                }

        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')

In [249]:
process_prices(content[1])

Error processing json: {'id': 'MLB4509988752', 'prices': [{'id': '15', 'type': 'standard', 'amount': 134.82, 'regular_amount': None, 'currency_id': 'BRL', 'last_updated': '2024-06-26T14:09:50Z', 'conditions': {'context_restrictions': [], 'start_time': None, 'end_time': None}}], 'item_id': 'MLB4509988752'}


In [252]:

price

NameError: name 'price' is not defined

In [275]:
content[5]

{'id': 'MLB3757145693',
 'prices': [{'id': '26',
   'type': 'standard',
   'amount': 249.99,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-09-09T12:00:20Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '25',
   'type': 'promotion',
   'amount': 169.99,
   'regular_amount': 199.99,
   'currency_id': 'BRL',
   'last_updated': '2024-09-07T03:32:33Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': '2024-09-01T03:00:00Z',
    'end_time': '2024-10-01T02:59:59Z'}}],
 'item_id': 'MLB3757145693'}

In [278]:
def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]

                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')
        

In [283]:
process_prices(content[8])

[{'item_id': 'MLB3788435711',
  'price_id': '10',
  'regular_amount': 399.0,
  'price': 239.4,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-07T04:59:44Z'},
 {'item_id': 'MLB3788435711',
  'price_id': '5',
  'regular_amount': None,
  'price': 199,
  'channel': 'channel_mshops',
  'last_updated': '2024-08-20T18:52:43Z'}]

In [282]:
content[6]

{'id': 'MLB3758665407',
 'prices': [{'id': '1',
   'type': 'standard',
   'amount': 159.1,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-06-27T19:09:48Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '45',
   'type': 'promotion',
   'amount': 151.14,
   'regular_amount': 159.1,
   'currency_id': 'BRL',
   'last_updated': '2024-09-18T18:35:39Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': '2024-09-01T03:00:00Z',
    'end_time': '2024-10-01T02:59:59Z'}}],
 'item_id': 'MLB3758665407'}

In [345]:
df = pd.DataFrame({'a': ['aaaa_bbbb','aaaa_bbbb']})
df['a'].apply(lambda x : x.split('_')[-1])

0    bbbb
1    bbbb
Name: a, dtype: object

In [351]:
df['a'].apply(lambda x : x.replace('aaaa', ''))

0    _bbbb
1    _bbbb
Name: a, dtype: object

In [354]:
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_COSTS

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Getting params to see costs
    query = f'''
        with items_details as (
        select distinct
            item_id,
            listing_type,
            category_id
        from datalake-v2-424516.datalake_v2.items_details
        where
            1=1
            and date(correspondent_date) = current_date()
            and seller_id = {seller_id}
        )

        select 
        p.item_id,
        d.listing_type,
        d.category_id,
        p.price,
        p.channel
        from datalake-v2-424516.datalake_v2.items_prices p
        inner join items_details d
        on p.item_id = d.item_id
        where 
        1=1
        and date(p.correspondent_date) = current_date()
    '''

    df_params = bigquery.run_query(query)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


[{'item_id': 'MLB3636253863',
  'listing_type': 'gold_special',
  'category_id': 'MLB455251',
  'price': 795.0,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3326149829',
  'listing_type': 'gold_special',
  'category_id': 'MLB7073',
  'price': 178.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3649200017',
  'listing_type': 'gold_special',
  'category_id': 'MLB7070',
  'price': 89.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB2059268709',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 78.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3775717904',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 799.2,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3204753274',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 297.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3474827016',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 152.63,
  'channel': 'channel_mar

In [363]:
type(df_params.to_dict(orient='records'))

list

In [364]:
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names=['id','price', 'category_id', 'listing_type_id']
)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=0__total_items=100__process_time=2024-09-21T15:24:24.192832-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=1__total_items=100__process_time=2024-09-21T15:24:24.540762-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=2__total_items=100__process_time=2024-09-21T15:24:24.955516-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=3__total_items=100__process_time=2024-09-21T15:24:24.977460-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=4__total_items=100__process_time=2024-09-21T15:24:24.490120-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=5__total_items=100__process_time=2024-09-21T15:24:24.709973-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=

In [378]:
params = {"id": "MLB3844407705",
      "listing_type_id": "gold_special",
      "category_id": "MLB269928",
      "price": 139.64,
      "channel": "marketplace"}

In [379]:
url = 'https://api.mercadolibre.com/sites/MLB/listing_prices'

response = requests.get(url, headers=headers, params=params)
response.json()

{'currency_id': 'BRL',
 'free_relist': False,
 'listing_exposure': 'highest',
 'listing_fee_amount': 0,
 'listing_fee_details': {'fixed_fee': 0, 'gross_amount': 0},
 'listing_type_id': 'gold_special',
 'listing_type_name': 'Clássico',
 'requires_picture': True,
 'sale_fee_amount': 16.06,
 'sale_fee_details': {'fixed_fee': 0,
  'gross_amount': 16.06,
  'percentage_fee': 11.5},
 'stop_time': '2044-09-16T00:00:00.000-04:00'}

In [381]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process_details, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests

data = {}

client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = 'gw'
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_DETAILS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/items/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names='results'
)

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: gw shop/meli/api_response/items/date=2024-09-22/total_items=2250__processing-date=2024-09-22T09:10:10.916974-03:00.json


['MLB3102014839',
 'MLB3185991384',
 'MLB3518556440',
 'MLB3102158572',
 'MLB1980290862',
 'MLB3128350046',
 'MLB3490815617',
 'MLB3455347127',
 'MLB3307239303',
 'MLB3360546777',
 'MLB3404650269',
 'MLB4045082674',
 'MLB1645130378',
 'MLB3128745411',
 'MLB3718223254',
 'MLB3294747925',
 'MLB3446177809',
 'MLB3102210265',
 'MLB3214213147',
 'MLB3109814507',
 'MLB3430645279',
 'MLB3140511025',
 'MLB3292759085',
 'MLB3525707162',
 'MLB3515213365',
 'MLB4188301548',
 'MLB3101824936',
 'MLB3214213127',
 'MLB3505588706',
 'MLB3296937609',
 'MLB3467248095',
 'MLB3520024632',
 'MLB1528437046',
 'MLB3102210302',
 'MLB3101735839',
 'MLB3188258649',
 'MLB1669426039',
 'MLB3102126308',
 'MLB3102162405',
 'MLB3420303951',
 'MLB3101742019',
 'MLB3101792569',
 'MLB3494724463',
 'MLB3103804576',
 'MLB3214213106',
 'MLB3102226551',
 'MLB3214391840',
 'MLB4085089798',
 'MLB1644491608',
 'MLB1529486139',
 'MLB3337224695',
 'MLB4087321888',
 'MLB1529503024',
 'MLB1849616885',
 'MLB3300679189',
 'MLB34947

In [2]:
import requests

In [3]:
client_id = '2951712600123976'
client_secret = 'QprAIl8ydXzcxFVHjnIHT6fUQ8KpzADV'
access_token = 'TG-66d96db3f899320001e32261-569119547'

In [4]:
url = "https://api.mercadolibre.com/oauth/token"
payload = {
    "grant_type": "refresh_token",
    "client_id": f"{client_id}",
    "client_secret": f"{client_secret}",
    "refresh_token": f"{access_token}"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
response = requests.post(url, data=payload, headers=headers)
tokens = response.json()
access_token = tokens.get("access_token")
new_refresh_token = tokens.get("refresh_token")
access_token
seller_id = '569119547'

In [5]:
def get_seller_id_and_store_name(client_id, client_secret, access_token):
    
    if not access_token:
        print("Getting access_token")
        token_url = 'https://api.mercadolibre.com/oauth/token'

        token_data = {
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret
        }

        response = requests.post(token_url, data=token_data)
        token_info = response.json()
        access_token = token_info['access_token']
    
    # Step 2: Retrieve User Information
    user_info_url = 'https://api.mercadolibre.com/users/me'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    user_response = requests.get(user_info_url, headers=headers)
    user_info = user_response.json()
    
    # Extract seller ID and store name
    seller_id = user_info['id']
    store_name = user_info.get('nickname', 'N/A').split('.')[0]  # Using 'nickname' as store name

    return store_name, seller_id

get_seller_id_and_store_name(client_id, client_secret, access_token)

('GW SHOP', 569119547)

In [434]:
url = "https://api.mercadolibre.com/oauth/token"
payload = {
    "grant_type": "refresh_token",
    "client_id": f"{client_id}",
    "client_secret": f"{client_secret}",
    "refresh_token": f"{access_token}"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
response = requests.post(url, data=payload, headers=headers)
tokens = response.json()
access_token = tokens.get("access_token")
new_refresh_token = tokens.get("refresh_token")
access_token
seller_id = '569119547'

In [6]:
from src.config import settings
url=settings.URL_ITEMS

In [11]:
all_products

['MLB3490815617',
 'MLB4057180066',
 'MLB3446178745',
 'MLB4151602254',
 'MLB3676506459',
 'MLB4700255102',
 'MLB4579745430',
 'MLB3646978165',
 'MLB4702983542',
 'MLB4700268106',
 'MLB3713231079',
 'MLB4702983566',
 'MLB3721298997',
 'MLB4198124396',
 'MLB3749755421',
 'MLB4825935042',
 'MLB4087334556',
 'MLB4063355328',
 'MLB3623362781',
 'MLB4693910870',
 'MLB4439026136',
 'MLB3695418243',
 'MLB4543891664',
 'MLB4818131796',
 'MLB4205697934',
 'MLB4044226286',
 'MLB3650275037',
 'MLB4311477538',
 'MLB4044196458',
 'MLB4346464598',
 'MLB3646810125',
 'MLB3654125065',
 'MLB4438963310',
 'MLB4450897088',
 'MLB4044213846',
 'MLB1527617749',
 'MLB1528461262',
 'MLB1528451289',
 'MLB1528437046',
 'MLB1528396545',
 'MLB1529520832',
 'MLB1529515421',
 'MLB1529513906',
 'MLB1529512379',
 'MLB1529503024',
 'MLB1529501027',
 'MLB1529491640',
 'MLB1529487337',
 'MLB1529486139',
 'MLB1529436862',
 'MLB1549097164',
 'MLB1562507089',
 'MLB1562503347',
 'MLB1562489323',
 'MLB1562488820',
 'MLB15624

In [56]:
url(seller_id)

'https://api.mercadolibre.com/users/569119547/items/search'

In [7]:
import requests

def fetch_items_beyond_1000(url, seller_id, access_token):
    url = url(seller_id)
    params = {
        'access_token': access_token,
        'limit': 50,  # You can set this up to 100
        'search_type': 'scan'
    }
    
    # First request
    response = requests.get(url, params=params)
    data = response.json()

    # Get the initial scroll_id from the response
    scroll_id = data.get('scroll_id', None)
    all_items = data.get('results', [])

    all_responses = []


    while scroll_id:
        # Update the request to use the scroll_id
        params.update({'scroll_id': scroll_id})
        
        # Request the next page using scroll_id
        response = requests.get(url, params=params)
        data = response.json()
        all_responses.append(data)

        # Add the new items to the list
        all_items.extend(data.get('results', []))
        
        # Get the next scroll_id (if any)
        scroll_id = data.get('scroll_id', None)
        
        if not data.get('results'):  # Stop when no more results
            break

    print(f'** Items found: {len(all_items)} **')
    return all_responses, all_items

all_responses, all_items = fetch_items_beyond_1000(url,seller_id, access_token)

** Items found: 100 **
** Items found: 150 **
** Items found: 200 **
** Items found: 250 **
** Items found: 300 **
** Items found: 350 **
** Items found: 400 **
** Items found: 450 **
** Items found: 500 **
** Items found: 550 **
** Items found: 600 **
** Items found: 650 **
** Items found: 700 **
** Items found: 750 **
** Items found: 800 **
** Items found: 850 **
** Items found: 900 **
** Items found: 950 **
** Items found: 1000 **
** Items found: 1050 **
** Items found: 1100 **
** Items found: 1150 **
** Items found: 1200 **
** Items found: 1250 **
** Items found: 1300 **
** Items found: 1350 **
** Items found: 1400 **
** Items found: 1450 **
** Items found: 1500 **
** Items found: 1550 **
** Items found: 1600 **
** Items found: 1650 **
** Items found: 1700 **
** Items found: 1750 **
** Items found: 1800 **
** Items found: 1850 **
** Items found: 1900 **
** Items found: 1950 **
** Items found: 2000 **
** Items found: 2050 **
** Items found: 2100 **
** Items found: 2150 **
** Items f

In [10]:
all_responses

[{'scroll_id': 'eyJpZCI6Ik1MQjE2MzIzMzAyODkiLCJudW1lcmljX2lkIjoxNjMyMzMwMjg5LCJzdG9wX3RpbWUiOiIyMDQwLTExLTMwVDE0OjI5OjE2LjAwMFoifQ==',
  'seller_id': '569119547',
  'results': ['MLB1549097164',
   'MLB1562507089',
   'MLB1562503347',
   'MLB1562489323',
   'MLB1562488820',
   'MLB1562476950',
   'MLB1562463919',
   'MLB1562512819',
   'MLB1529563064',
   'MLB1576620547',
   'MLB1576616272',
   'MLB1588552137',
   'MLB1588548553',
   'MLB1588546167',
   'MLB1588535618',
   'MLB1588533079',
   'MLB1528323245',
   'MLB1587989889',
   'MLB1605086033',
   'MLB1529441868',
   'MLB1628608642',
   'MLB1632365490',
   'MLB1645136446',
   'MLB1645130378',
   'MLB1644931120',
   'MLB1644927650',
   'MLB1644793358',
   'MLB1644500193',
   'MLB1644495919',
   'MLB1644491608',
   'MLB1646194911',
   'MLB1646190992',
   'MLB1646178298',
   'MLB1645440803',
   'MLB1645440650',
   'MLB1645431148',
   'MLB1645427421',
   'MLB1587980713',
   'MLB1658042809',
   'MLB1669426039',
   'MLB1668888765',
   'ML

In [9]:
import pandas as pd 

pd.Series(all_items).nunique()

2204

In [47]:
limit = 60 
params = {
    'access_token': access_token,
    'limit': limit,
    'offset':900,
    'adsfqgfr': 2
}
response = requests.get(url(seller_id), params=params)

data = response.json()
print(data)
items = data['results']
items[:2]


{'seller_id': '569119547', 'results': ['MLB3464802311', 'MLB3464705707', 'MLB3467999913', 'MLB3467579179', 'MLB3467272683', 'MLB3467259941', 'MLB3467248095', 'MLB3467248089', 'MLB3467198293', 'MLB3467186301', 'MLB3467173459', 'MLB4100143166', 'MLB4100142180', 'MLB4100066696', 'MLB4100065624', 'MLB4099501842', 'MLB4099481842', 'MLB4099477134', 'MLB3468434519', 'MLB3468430559', 'MLB3468417763', 'MLB3468416047', 'MLB3468410079', 'MLB3468371201', 'MLB3468359021', 'MLB3468356689', 'MLB4119285612', 'MLB4119272894', 'MLB4128947182', 'MLB4127934778', 'MLB4138903802', 'MLB4138890942', 'MLB4138851262', 'MLB3485317489', 'MLB3485304551', 'MLB3485304539', 'MLB3485291935', 'MLB3485291907', 'MLB3485291895', 'MLB3485280485', 'MLB3485267491', 'MLB3485267487', 'MLB3485241959', 'MLB3485241941', 'MLB3485218357', 'MLB4141255652', 'MLB3487697373', 'MLB3296752135', 'MLB4087385854', 'MLB3491245597', 'MLB4156462910', 'MLB4155383178', 'MLB4155217232', 'MLB3492063411', 'MLB3492029497', 'MLB3492022389', 'MLB34939

['MLB3464802311', 'MLB3464705707']

In [42]:
data

{'seller_id': '569119547',
 'results': ['MLB3464802311',
  'MLB3464705707',
  'MLB3467999913',
  'MLB3467579179',
  'MLB3467272683',
  'MLB3467259941',
  'MLB3467248095',
  'MLB3467248089',
  'MLB3467198293',
  'MLB3467186301',
  'MLB3467173459',
  'MLB4100143166',
  'MLB4100142180',
  'MLB4100066696',
  'MLB4100065624',
  'MLB4099501842',
  'MLB4099481842',
  'MLB4099477134',
  'MLB3468434519',
  'MLB3468430559',
  'MLB3468417763',
  'MLB3468416047',
  'MLB3468410079',
  'MLB3468371201',
  'MLB3468359021',
  'MLB3468356689',
  'MLB4119285612',
  'MLB4119272894',
  'MLB4128947182',
  'MLB4127934778',
  'MLB4138903802',
  'MLB4138890942',
  'MLB4138851262',
  'MLB3485317489',
  'MLB3485304551',
  'MLB3485304539',
  'MLB3485291935',
  'MLB3485291907',
  'MLB3485291895',
  'MLB3485280485',
  'MLB3485267491',
  'MLB3485267487',
  'MLB3485241959',
  'MLB3485241941',
  'MLB3485218357',
  'MLB4141255652',
  'MLB3487697373',
  'MLB3296752135',
  'MLB4087385854',
  'MLB3491245597',
  'MLB415646

In [52]:
import requests

def fetch_all_items(url, access_token, seller_id):
    offset = 0
    page = 1  # Page parameter (used if needed)
    max_offset = 1000  # MercadoLibre API max offset
    count_total_results = 0
    all_products = []
    all_responses = []  # List to store all the JSON responses
    limit = 50  # Number of products per request
    scan_mode = False  # To track if scan mode is enabled
    scroll_id = None  # Store scroll_id for scan mode

    params = {
        'access_token': access_token,
        'limit': limit,
    }

    while True:
        if scan_mode:
            # Switch to scan mode pagination after hitting max offset
            params['scroll_id'] = scroll_id
            params['search_type'] = 'scan'
            if 'offset' in params:
                del params['offset']  # Remove offset when using scan mode
            print(f"Switching to scan mode with scroll_id {scroll_id}")
        elif offset >= max_offset:
            # Use page parameter if needed (optional for MercadoLibre API)
            params['page'] = page
            if 'offset' in params:
                del params['offset']
            print(f"Switching to pagination at page {page}")
        else:
            # Use offset for first 1000 items
            params['offset'] = offset

        # Request API
        response = requests.get(url(seller_id), params=params)

        # Check request status
        if response.status_code != 200:
            raise Exception(f"Failed to fetch data: {response.json()}")

        # Get items data
        items_data = response.json()
        results = items_data['results']
        print(f"Offset {offset} (or page {page}): {len(results)} items")

        # Add the entire JSON response to the list
        all_responses.append(items_data)

        # Increment count of total results
        count_total_results += len(results)

        # Add the current batch of product IDs to the all_products list
        products = items_data.get('results', [])
        all_products.extend(products)

        # Stop condition: all daily information has been imported
        if count_total_results >= items_data['paging']['total']:
            print(f'** Request items have been processed. **')
            break

        # Update offset and page
        if not scan_mode:
            if offset < max_offset:
                offset += limit
            else:
                page += 1

            # If we've reached the max offset, switch to scan mode
            if offset >= max_offset:
                scan_mode = True
                # Get the initial scroll_id from the first response in scan mode
                scroll_id = items_data.get('scroll_id')
                print(f"Scan mode activated with scroll_id: {scroll_id}")
        else:
            # In scan mode, use the scroll_id from the previous response
            scroll_id = items_data.get('scroll_id', None)

            # If no more scroll_id, exit the loop
            if not scroll_id or not results:
                print(f'** No more results to scan. **')
                break

    print(f'** Number of items found: {len(all_products)}')
    
    return all_products, all_responses


all_product, all_responses = fetch_all_items(url, access_token, seller_id)

Offset 0 (or page 1): 50 items
Offset 50 (or page 1): 50 items
Offset 100 (or page 1): 50 items
Offset 150 (or page 1): 50 items
Offset 200 (or page 1): 50 items
Offset 250 (or page 1): 50 items
Offset 300 (or page 1): 50 items
Offset 350 (or page 1): 50 items
Offset 400 (or page 1): 50 items
Offset 450 (or page 1): 50 items
Offset 500 (or page 1): 50 items
Offset 550 (or page 1): 50 items
Offset 600 (or page 1): 50 items
Offset 650 (or page 1): 50 items
Offset 700 (or page 1): 50 items
Offset 750 (or page 1): 50 items
Offset 800 (or page 1): 50 items
Offset 850 (or page 1): 50 items
Offset 900 (or page 1): 50 items
Offset 950 (or page 1): 50 items
Scan mode activated with scroll_id: None
Switching to scan mode with scroll_id None
Offset 1000 (or page 1): 50 items
Switching to scan mode with scroll_id eyJpZCI6Ik1MQjE1Mjk0MzY4NjIiLCJudW1lcmljX2lkIjoxNTI5NDM2ODYyLCJzdG9wX3RpbWUiOiIyMDQwLTA1LTE1VDA0OjAwOjAwLjAwMFoifQ==
Offset 1000 (or page 1): 50 items
Switching to scan mode with scroll_i

In [54]:
len(all_product)

2250

In [1]:
import pandas as pd
pd.Series(all_products).nunique()

NameError: name 'all_products' is not defined

In [435]:
access_token

'APP_USR-2951712600123976-092217-603d352f2fa7a2aaa722234b4ed93d8d-354359142'

In [457]:
url = settings.URL_ITEMS
url(seller_id)

'https://api.mercadolibre.com/users/569119547/items/search'

In [456]:
url(seller_id)
access_token

TypeError: 'str' object is not callable

In [458]:
params = {
  'access_token': access_token,
  'limit': limit,
}
  
response = requests.get(url(seller_id), params=params)
response.json()

{'seller_id': '569119547',
 'results': ['MLB4057180066',
  'MLB3446178745',
  'MLB4151602254',
  'MLB3676506459',
  'MLB4700255102',
  'MLB4579745430',
  'MLB3646978165',
  'MLB4702983542',
  'MLB4700268106',
  'MLB3713231079',
  'MLB4702983566',
  'MLB3721298997',
  'MLB4198124396',
  'MLB3749755421',
  'MLB4825935042',
  'MLB4087334556',
  'MLB4063355328',
  'MLB3623362781',
  'MLB4693910870',
  'MLB4439026136',
  'MLB3695418243',
  'MLB4543891664',
  'MLB4818131796',
  'MLB4205697934',
  'MLB4044226286',
  'MLB3650275037',
  'MLB4311477538',
  'MLB4044196458',
  'MLB4346464598',
  'MLB3646810125',
  'MLB3654125065',
  'MLB4438963310',
  'MLB4450897088',
  'MLB4044213846',
  'MLB1527617749',
  'MLB1528461262',
  'MLB1528451289',
  'MLB1528437046',
  'MLB1528396545',
  'MLB1529520832',
  'MLB1529515421',
  'MLB1529513906',
  'MLB1529512379',
  'MLB1529503024',
  'MLB1529501027',
  'MLB1529491640',
  'MLB1529487337',
  'MLB1529486139',
  'MLB1529436862',
  'MLB1529414028'],
 'paging': 

In [ ]:
url = settings.URL_ITEMS

all_products, all_responses = fetch_all_items(url,access_token, seller_id = '569119547',)

In [405]:
pd.Series(all_products)


0       MLB3490815617
1       MLB4057180066
2       MLB3446178745
3       MLB4151602254
4       MLB3676506459
            ...      
2245    MLB1529491640
2246    MLB1529487337
2247    MLB1529486139
2248    MLB1529436862
2249    MLB1529414028
Length: 2250, dtype: object